In [1]:

!pip3 install -q  wandb
!pip install -q pyarrow==18.*
!pip install -q ipywidgets==8.1.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 41.1 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
pandas-gbq 0.26.1 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [ ]:
pip install -q unsloth

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     llm_int8_enable_fp32_cpu_offload=True
# )

## Importing the synthetic_pii_finance_multilingual dataset from HuggingFace

In [1]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/gretelai/synthetic_pii_finance_multilingual/" + splits["train"])

## Dataset Overview

In [2]:
df.head()

,level_0,index,document_type,document_description,expanded_type,expanded_description,language,language_description,domain,generated_text,pii_spans,conformance_score,quality_score,toxicity_score,bias_score,groundedness_score
0,40012,40012,Supply Chain Management Agreement,A legal contract outlining the terms and condi...,Vendor Management Contract,This subtype involves the contractual agreemen...,English,English language as spoken in the United State...,finance,SUPPLY CHAIN MANAGEMENT AGREEMENT\n\nThis Supp...,"[{""start"": 119, ""end"": 141, ""label"": ""date""}, ...",85,90,5,15,95
1,46425,46425,Supply Chain Management Agreement,A legal contract outlining the terms and condi...,Supply Chain Resilience Framework,This subtype details the framework for buildin...,English,English language as spoken in the United State...,finance,SUPPLY CHAIN RESILIENCE FRAMEWORK\n\nThis Supp...,"[{""start"": 119, ""end"": 142, ""label"": ""date""}, ...",92,87,5,12,95
2,4689,4689,Real Estate Loan Agreement,A legal contract outlining terms and condition...,International Real Estate Investment Loan Agre...,This subtype encompasses loans for internation...,Spanish,Spanish language as spoken in Spain or Mexico,finance,CONTRATO DE PRÉSTAMO PARA INVERSIÓN INMOBILIAR...,"[{""start"": 182, ""end"": 209, ""label"": ""street_a...",85,90,5,15,95
3,3002,3002,Real Estate Loan Agreement,A legal contract outlining terms and condition...,Commercial Property Loan Contract,This subtype focuses on loans for commercial r...,Italian,Italian language as spoken in Italy,finance,REPUBBLICA ITALIANA\n\nCONTRATTO DI PRESTITO I...,"[{""start"": 85, ""end"": 103, ""label"": ""name""}, {...",85,90,5,10,95
4,16187,16187,Email,A communication sent electronically containing...,Invitation,Create an email inviting recipients to an even...,France,French language as spoken in France,finance,Subject: Invitation à notre soirée de lancemen...,"[{""start"": 202, ""end"": 207, ""label"": ""time""}, ...",85,90,5,15,95


## The Fields which we will use for finetununf are 'generated_text'  and 'pii_spans'.
## We will convert the text into this following format for finetuning
```python
SUPPLY CHAIN MANAGEMENT AGREEMENT

This Supply Chain Management Agreement (the "Agreement") is entered into as of this <DATE>1st day of March, 2021</DATE> (the "Effective Date"), by and between <COMPANY>Cameron-Mcknight</COMPANY>, a company organized and existing under the laws of the state of Delaware, with its head office located at <STREET_ADDRESS>81685 Lopez Lodge, Apt. 6502</STREET_ADDRESS>, hereinafter referred to as "<COMPANY>Cameron-Mcknight</COMPANY>", and <NAME>Jann N. Butte</NAME>, an individual with a mailing address of <STREET_ADDRESS>81685 Lopez Lodge, Apt. 6502</STREET_ADDRESS>.

WHEREAS, <COMPANY>Cameron-Mcknight</COMPANY> desires to engage the services of Vendor for the provision of goods and services in connection with <COMPANY>Cameron-Mcknight</COMPANY>'s supply chain operations; and

WHEREAS, Vendor desires to provide such goods and services to <COMPANY>Cameron-Mcknight</COMPANY> on the terms and conditions set forth herein.

```

In [3]:
import json
def add_ner_tags(text, detections):
    # Sort detections by start_position in descending order
    detections = sorted(detections, key=lambda x: x["start"], reverse=True)
    
    for entity in detections:
        start = entity["start"]
        end = entity["end"]
        entity_type = entity["label"].upper()
        
        # Insert end tag first, then start tag
        text = text[:end] + f"</{entity_type}>" + text[end:]
        text = text[:start] + f"<{entity_type}>" + text[start:]
    
    return text

In [4]:
df['tagged_text'] = df.apply(lambda row: add_ner_tags(row['generated_text'], json.loads(row['pii_spans'])), axis=1)


In [6]:
print(df['tagged_text'][0])

SUPPLY CHAIN MANAGEMENT AGREEMENT

This Supply Chain Management Agreement (the "Agreement") is entered into as of this <DATE>1st day of March, 2021</DATE> (the "Effective Date"), by and between <COMPANY>Cameron-Mcknight</COMPANY>, a company organized and existing under the laws of the state of Delaware, with its head office located at <STREET_ADDRESS>81685 Lopez Lodge, Apt. 6502</STREET_ADDRESS>, hereinafter referred to as "<COMPANY>Cameron-Mcknight</COMPANY>", and <NAME>Jann N. Butte</NAME>, an individual with a mailing address of <STREET_ADDRESS>81685 Lopez Lodge, Apt. 6502</STREET_ADDRESS>.

WHEREAS, <COMPANY>Cameron-Mcknight</COMPANY> desires to engage the services of Vendor for the provision of goods and services in connection with <COMPANY>Cameron-Mcknight</COMPANY>'s supply chain operations; and

WHEREAS, Vendor desires to provide such goods and services to <COMPANY>Cameron-Mcknight</COMPANY> on the terms and conditions set forth herein.

NOW, THEREFORE, in consideration of the 

## Now we will extract the unique entities available in the entire dataset and add a description for each entity 
## This will be passed to the llm as a Prompt

In [ ]:
def get_unique_entities(df):
    unique_entities = set()
    
    for spans_str in df['pii_spans']:
        try:
            spans = json.loads(spans_str)
            for entity in spans:
                unique_entities.add(entity['label'].upper())
        except (json.JSONDecodeError, TypeError):
            continue
    return unique_entities

In [ ]:
unique_entities = get_unique_entities(df)

In [6]:
entity_description_dict = {
    "ACCOUNT_PIN": "Personal Identification Number for accounts",
    "API_KEY": "Authentication token for accessing APIs",
    "BANK_ROUTING_NUMBER": "Bank’s routing number used in financial transactions",
    "BBAN": "Basic Bank Account Number used in international banking",
    "COMPANY": "Name of a business or organization",
    "CREDIT_CARD_NUMBER": "Number on a credit card used for payments",
    "CREDIT_CARD_SECURITY_CODE": "CVV or security code on a credit card",
    "CUSTOMER_ID": "Unique identifier assigned to a customer",
    "DATE": "Calendar date (e.g. 2025-04-10)",
    "DATE_OF_BIRTH": "Person’s date of birth",
    "DRIVER_LICENSE_NUMBER": "License number assigned to a driver",
    "EMAIL": "Email address",
    "EMPLOYEE_ID": "Identifier assigned to an employee",
    "FIRST_NAME": "Given name of a person",
    "IBAN": "International Bank Account Number used in global banking",
    "IPV4": "IPv4 format IP address (e.g. 192.168.1.1)",
    "IPV6": "IPv6 format IP address",
    "LAST_NAME": "Surname or family name of a person",
    "LOCAL_LATLNG": "Geolocation data in latitude and longitude format",
    "NAME": "Full name of a person",
    "PASSPORT_NUMBER": "Number identifying a person’s passport",
    "PASSWORD": "Confidential login credential",
    "SSN": "Social Security Number (often US)",
    "STREET_ADDRESS": "Street-level residential or business address",
    "SWIFT_BIC_CODE": "Bank Identifier Code used for international wires",
    "TIME": "Time of day (e.g. 14:00, 2 PM)",
    "USER_NAME": "Username used for login or identification"}

## Importing the Unsloth llama model for fine tuning . Unsloth is Opensource Library for finetuning

In [6]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("HF_TOKEN")


# import bitsandbytes

# from transformers import AutoTokenizer, AutoModelForCausalLM, AutoProcessor


# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct",token=secret_value_0)
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct",token=secret_value_0,
#                                              device_map="auto",
#                                              quantization_config = bnb_config)
# processor = AutoProcessor.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
from datasets import Dataset
from unsloth import is_bfloat16_supported

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length=8192,
    load_in_4bit=True,
    dtype=None,
    gpu_memory_utilization=0.9
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-04-12 06:09:27.899974: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744438168.132798      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744438168.198543      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.2",  # or "chatml", "alpaca", etc. based on how you finetuned
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    map_eos_token=True
)

In [7]:
import os
import torch
torch.cuda.empty_cache() 
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "max_split_size_mb:100"

## I am goining to use instruction based fine tuning. For that I am using Chain-Of-Thought in the prompt design for instructing the LLM

In [ ]:
# First user message
usr_msg1 = f"""You are given a user utterance that may contain Personal Identifiable Information (PII). 
        You are also given a list of entity types representing Personal Identifiable Information (PII). 
        Your task is to detect and identify all instances of the supplied PII entity types in the user utterance. 
        The output must have the same content as the input. Only the tokens that match the PII entities in the 
        list should be enclosed within XML tags. The XML tag comes from the PII entities described in the list below. 
        For example, a given name of a person should be enclosed within <FIRST_NAME></FIRST_NAME> tags.
        Ensure that all entities are identified. Do not perform false identifications.
        \n\nList Of Entities\n{json.dumps(description_dict,indent=2)}
        \n\n
        Are the instructions clear to you?"""

# First assistant message which is a reply to usr_msg1
# I hardcode this msg once the model produced a resonably good response.
# Note that the response comes from a non-fine-tuned version of the model.
# The idea is to sample a good 'thought' from the base version of the model.    
asst_msg1 = """Yes, the instructions are clear. I will identify and enclose within the corresponding XML tags, 
    all instances of the specified PII entity types in the user utterance. For example, 
    <FIRST_NAME><Given name of a person></FIRST_NAME>, <PASSPORT_NUMBER><Number identifying a person’s passport></PASSPORT_NUMBER>, etc. 
    leaving the rest of the user utterance unchanged."""

# Here I hardcode a few shot example as a user message
usr_msg2 = "My name is John Doe, and I can be contacted at meet_me_john@gmail.com"

# Here I hardcode the appropriate response from the assitant as the
# correct output of the few shot example
asst_msg2 = "My name is <FIRST_NAME>John</FIRST_NAME> <LAST_NAME>Doe</LAST_NAME>, and I can be contacted at <EMAIL>meet_me_john@gmail.com</EMAIL>"

# Here I ask the assistant why its response to the last user message was the 
# correct response
usr_msg3 = "Give a brief explanation of why your answer is correct."




In [ ]:
messages = [
        {"role": "user", "content": usr_msg1},
        {"role": "assistant", "content": asst_msg1},
        {"role": "user", "content": usr_msg2},
        {"role": "assistant", "content": asst_msg2},
        {"role": "user", "content": usr_msg3},
    ]

In [ ]:
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True,return_tensors="pt",add_generation_prompt=True)
outputs = model.generate(tokenized_chat, max_new_tokens=512) 
output_text = tokenizer.decode(outputs[0])

In [ ]:
# This is the response I got from the assistant . 
# I have Hardcoded this into the prompt generating function
asst_msg3 = re.findall(r'<\|end_header_id\|>(.*?)<\|eot_id\|>', asst_msg3, re.DOTALL)[-1]

In [8]:
print(asst_msg3)

My answer is correct because I identified the specified PII entity types in the user utterance and enclosed them within the corresponding XML tags.

                    - "John" is a given name, so it was enclosed within <FIRST_NAME> tags.
                    - "Doe" is a surname, so it was enclosed within <LAST_NAME> tags.
                    - "meet_me_john@gmail.com" is an email address, so it was enclosed within <EMAIL> tags.


## The function which creates the entire training data. 
## The data looks like this
```text
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 10 Apr 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

You are given a user utterance that may contain Personal Identifiable Information (PII). 
        You are also given a list of entity types representing Personal Identifiable Information (PII). 
        Your task is to detect and identify all instances of the supplied PII entity types in the user utterance. 
        The output must have the same content as the input. Only the tokens that match the PII entities in the 
        list should be enclosed within XML tags. The XML tag comes from the PII entities described in the list below. 
        For example, a given name of a person should be enclosed within <FIRST_NAME></FIRST_NAME> tags.
        Ensure that all entities are identified. Do not perform false identifications.
        

List Of Entities
{
  "ACCOUNT_PIN": "Personal Identification Number for accounts",
  "API_KEY": "Authentication token for accessing APIs",
  "BANK_ROUTING_NUMBER": "Bank\u2019s routing number used in financial transactions",
  "BBAN": "Basic Bank Account Number used in international banking",
  "COMPANY": "Name of a business or organization",
  "CREDIT_CARD_NUMBER": "Number on a credit card used for payments",
  "CREDIT_CARD_SECURITY_CODE": "CVV or security code on a credit card",
  "CUSTOMER_ID": "Unique identifier assigned to a customer",
  "DATE": "Calendar date (e.g. 2025-04-10)",
  "DATE_OF_BIRTH": "Person\u2019s date of birth",
  "DRIVER_LICENSE_NUMBER": "License number assigned to a driver",
  "EMAIL": "Email address",
  "EMPLOYEE_ID": "Identifier assigned to an employee",
  "FIRST_NAME": "Given name of a person",
  "IBAN": "International Bank Account Number used in global banking",
  "IPV4": "IPv4 format IP address (e.g. 192.168.1.1)",
  "IPV6": "IPv6 format IP address",
  "LAST_NAME": "Surname or family name of a person",
  "LOCAL_LATLNG": "Geolocation data in latitude and longitude format",
  "NAME": "Full name of a person",
  "PASSPORT_NUMBER": "Number identifying a person\u2019s passport",
  "PASSWORD": "Confidential login credential",
  "SSN": "Social Security Number (often US)",
  "STREET_ADDRESS": "Street-level residential or business address",
  "SWIFT_BIC_CODE": "Bank Identifier Code used for international wires",
  "TIME": "Time of day (e.g. 14:00, 2 PM)",
  "USER_NAME": "Username used for login or identification"
}
        


        Are the instructions clear to you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Yes, the instructions are clear. I will identify and enclose within the corresponding XML tags, 
        all instances of the specified PII entity types in the user utterance. For example, 
        <FIRST_NAME><Given name of a person></FIRST_NAME>, <PASSPORT_NUMBER><Number identifying a person’s passport></PASSPORT_NUMBER>, etc. 
        leaving the rest of the user utterance unchanged.<|eot_id|><|start_header_id|>user<|end_header_id|>

My name is John Doe, and I can be contacted at meet_me_john@gmail.com<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My name is <FIRST_NAME>John</FIRST_NAME> <LAST_NAME>Doe</LAST_NAME>, and I can be contacted at <EMAIL>meet_me_john@gmail.com</EMAIL><|eot_id|><|start_header_id|>user<|end_header_id|>

Give a brief explanation of why your answer is correct.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

My answer is correct because I identified the specified PII entity types in the user utterance and enclosed them within the corresponding XML tags.

                    - "John" is a given name, so it was enclosed within <FIRST_NAME> tags.
                    - "Doe" is a surname, so it was enclosed within <LAST_NAME> tags.
                    - "meet_me_john@gmail.com" is an email address, so it was enclosed within <EMAIL> tags.<|eot_id|><|start_header_id|>user<|end_header_id|>

Great! I am now going to give you another user utterance. Please detect PII entities in it according to the previous instructions. Do not include an explanation in your answer.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Sure! Please give me the user utterance.<|eot_id|><|start_header_id|>user<|end_header_id|>

SUPPLY CHAIN MANAGEMENT AGREEMENT

This Supply Chain Management Agreement (the "Agreement") is entered into as of this 1st day of March, 2021 (the "Effective Date"), by and between Cameron-Mcknight, a company organized and existing under the laws of the state of Delaware, with its head office located at 81685 Lopez Lodge, Apt. 6502, hereinafter referred to as "Cameron-Mcknight", and Jann N. Butte, an individual with a mailing address of 81685 Lopez Lodge, Apt. 6502.

WHEREAS, Cameron-Mcknight desires to engage the services of Vendor for the provision of goods and services in connection with Cameron-Mcknight's supply chain operations; and

WHEREAS, Vendor desires to provide such goods and services to Cameron-Mcknight on the terms and conditions set forth herein.

NOW, THEREFORE, in consideration of the mutual covenants and promises contained herein, the parties hereto agree as follows:

1. APPOINTMENT

Cameron-Mcknight hereby appoints Vendor as a vendor for the provision of goods and services in connection with Cameron-Mcknight's supply chain operations. Vendor shall provide such goods and services in accordance with the terms and conditions set forth herein.

2. TERM

This Agreement shall commence on the Effective Date and shall continue in force for a period of one (1) year, unless earlier terminated in accordance with the provisions hereof.

3. RESPONSIBILITIES OF VENDOR

3.1. Performance. Vendor shall perform its obligations under this Agreement in a professional and workmanlike manner in accordance with industry standards.

3.2. Compliance. Vendor shall comply with all applicable laws, rules, and regulations in connection with its performance under this Agreement.

3.3. Confidentiality. Vendor shall maintain the confidentiality of all confidential information of Cameron-Mcknight and shall not disclose such confidential information to any third party without the prior written consent of Cameron-Mcknight.

4. PERFORMANCE METRICS

Camer<|eot_id|><|start_header_id|>assistant<|end_header_id|>

SUPPLY CHAIN MANAGEMENT AGREEMENT

This Supply Chain Management Agreement (the "Agreement") is entered into as of this <DATE>1st day of March, 2021</DATE> (the "Effective Date"), by and between <COMPANY>Cameron-Mcknight</COMPANY>, a company organized and existing under the laws of the state of Delaware, with its head office located at <STREET_ADDRESS>81685 Lopez Lodge, Apt. 6502</STREET_ADDRESS>, hereinafter referred to as "<COMPANY>Cameron-Mcknight</COMPANY>", and <NAME>Jann N. Butte</NAME>, an individual with a mailing address of <STREET_ADDRESS>81685 Lopez Lodge, Apt. 6502</STREET_ADDRESS>.

WHEREAS, <COMPANY>Cameron-Mcknight</COMPANY> desires to engage the services of Vendor for the provision of goods and services in connection with <COMPANY>Cameron-Mcknight</COMPANY>'s supply chain operations; and

WHEREAS, Vendor desires to provide such goods and services to <COMPANY>Cameron-Mcknight</COMPANY> on the terms and conditions set forth herein.

NOW, THEREFORE, in consideration of the mutual covenants and promises contained herein, the parties hereto agree as follows:

1. APPOINTMENT

<COMPANY>Cameron-Mcknight</COMPANY> hereby appoints Vendor as a vendor for the provision of goods and services in connection with <COMPANY>Cameron-Mcknight</COMPANY>'s supply chain operations. Vendor shall provide such goods and services in accordance with the terms and conditions set forth herein.

2. TERM

This Agreement shall commence on the Effective Date and shall continue in force for a period of <DATE>one (1) year</DATE>, unless earlier terminated in accordance with the provisions hereof.

3. RESPONSIBILITIES OF VENDOR

3.1. Performance. Vendor shall perform its obligations under this Agreement in a professional and workmanlike manner in accordance with industry standards.

3.2. Compliance. Vendor shall comply with all applicable laws, rules, and regulations in connection with its performance under this Agreement.

3.3. Confidentiality. Vendor shall maintain the confidentiality of all confidential information of <COMPANY>Cameron-Mcknight</COMPANY> and shall not disclose such confidential information to any third party without the prior written consent of <COMPANY>Cameron-Mcknight</COMPANY>.

4. PERFORMANCE METRICS

Camer<|eot_id|>
```

In [ ]:
from  transformers import PreTrainedTokenizerBase
def get_fine_tune_prompt_xml(
    description_dict: dict, 
    input_str: str,
    label_str: str,
    tokenizer: PreTrainedTokenizerBase,
) -> torch.Tensor:

    """
    Args:
    description (dict): dict of strings representing entity labels and its
                          corresponding description
    input_str (str): Actual input string on which detections need to be
                     performed
    label_str (str): Expected output string corresponding to input_str
    tokenizer (PreTrainedTokenizerBase): A tokenizer corresponding to the model
                                         being fine-tuned

    Returns: 
    torch.Tensor: Tensor of tokenized input ids
    """

    usr_msg1 = f"""You are given a user utterance that may contain Personal Identifiable Information (PII). 
        You are also given a list of entity types representing Personal Identifiable Information (PII). 
        Your task is to detect and identify all instances of the supplied PII entity types in the user utterance. 
        The output must have the same content as the input. Only the tokens that match the PII entities in the 
        list should be enclosed within XML tags. The XML tag comes from the PII entities described in the list below. 
        For example, a given name of a person should be enclosed within <FIRST_NAME></FIRST_NAME> tags.
        Ensure that all entities are identified. Do not perform false identifications.
        \n\nList Of Entities\n{json.dumps(description_dict,indent=2)}
        \n\n
        Are the instructions clear to you?"""
    
    asst_msg1 = """Yes, the instructions are clear. I will identify and enclose within the corresponding XML tags, 
        all instances of the specified PII entity types in the user utterance. For example, 
        <FIRST_NAME><Given name of a person></FIRST_NAME>, <PASSPORT_NUMBER><Number identifying a person’s passport></PASSPORT_NUMBER>, etc. 
        leaving the rest of the user utterance unchanged."""
    
    usr_msg2 = "My name is John Doe, and I can be contacted at meet_me_john@gmail.com"
    
    asst_msg2 = "My name is <FIRST_NAME>John</FIRST_NAME> <LAST_NAME>Doe</LAST_NAME>, and I can be contacted at <EMAIL>meet_me_john@gmail.com</EMAIL>"
    
    usr_msg3 = "Give a brief explanation of why your answer is correct."


    asst_msg3 = """My answer is correct because I identified the specified PII entity types in the user utterance and enclosed them within the corresponding XML tags.

                    - "John" is a given name, so it was enclosed within <FIRST_NAME> tags.
                    - "Doe" is a surname, so it was enclosed within <LAST_NAME> tags.
                    - "meet_me_john@gmail.com" is an email address, so it was enclosed within <EMAIL> tags."""
    
    usr_msg4 = """Great! I am now going to give you another user utterance. Please detect PII entities in it according to the previous instructions. Do not include an explanation in your answer."""
    
    asst_msg4 = "Sure! Please give me the user utterance."

    messages = [
        {"role": "user", "content": usr_msg1},
        {"role": "assistant", "content": asst_msg1},
        {"role": "user", "content": usr_msg2},
        {"role": "assistant", "content": asst_msg2},
        {"role": "user", "content": usr_msg3},
        {"role": "assistant", "content": asst_msg3},
        {"role": "user", "content": usr_msg4},
        {"role": "assistant", "content": asst_msg4},
        {"role": "user", "content": input_str},
        {"role": "assistant", "content": label_str},
    ]
    
    encoded_input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt")

    return tokenizer.decode(encoded_input_ids[0])

In [ ]:
df['text'] = df.apply(lambda row: get_fine_tune_prompt_xml(entity_description_dict,row['generated_text'], row['tagged_text'],tokenizer), axis=1)


## Custom Loss Masking
- This will help the model to calculate the loss only on the generated part . not on the entire given input text

In [9]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from transformers.utils import PaddingStrategy
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union

@dataclass
class CustomDataCollatorWithPadding:
    """
    Data collator that will dynamically pad the inputs received.

    Args:
        tokenizer ([`PreTrainedTokenizer`] or [`PreTrainedTokenizerFast`]):
            The tokenizer used for encoding the data.
        padding (`bool`, `str` or [`~utils.PaddingStrategy`], *optional*, defaults to `True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:

            - `True` or `'longest'` (default): Pad to the longest sequence in the batch (or no padding if only a single
              sequence is provided).
            - `'max_length'`: Pad to a maximum length specified with the argument `max_length` or to the maximum
              acceptable input length for the model if that argument is not provided.
            - `False` or `'do_not_pad'`: No padding (i.e., can output a batch with sequences of different lengths).
        max_length (`int`, *optional*):
            Maximum length of the returned list and optionally padding length (see above).
        pad_to_multiple_of (`int`, *optional*):
            If set will pad the sequence to a multiple of the provided value.

            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
        return_tensors (`str`, *optional*, defaults to `"pt"`):
            The type of Tensor to return. Allowable values are "np", "pt" and "tf".
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    return_tensors: str = "pt"

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        batch = self.tokenizer.pad(
            features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )
        labels = batch["input_ids"].clone()
        
        # Set loss mask for all pad tokens
        labels[labels == self.tokenizer.pad_token_id] = -100
        
        # Compute loss mask for appropriate tokens only
        for i in range(batch['input_ids'].shape[0]):
            
            # Decode the training input
            text_content = self.tokenizer.decode(batch['input_ids'][i][1:])  # slicing from [1:] is important because tokenizer adds bos token
            
            # Extract substrings for prompt text in the training input
            # The training input ends at the last user msg ending in [/INST]
            prompt_gen_boundary = text_content.rfind("<|end_header_id|>") + len("<|end_header_id|>")
            prompt_text = text_content[:prompt_gen_boundary]
            
            # print(f"""PROMPT TEXT:\n{prompt_text}""")
            
            # retokenize the prompt text only
            prompt_text_tokenized = self.tokenizer(
                prompt_text,
                return_overflowing_tokens=False,
                return_length=False,
            )
            # compute index where prompt text ends in the training input
            prompt_tok_idx = len(prompt_text_tokenized['input_ids'])
            
            # Set loss mask for all tokens in prompt text
            labels[i][range(prompt_tok_idx)] = -100
            
            # print("================DEBUGGING INFORMATION===============")
            # for idx, tok in enumerate(labels[i]):
            #     token_id = batch['input_ids'][i][idx]
            #     decoded_token_id = self.tokenizer.decode(batch['input_ids'][i][idx])
            #     print(f"""TOKID: {token_id} | LABEL: {tok} || DECODED: {decoded_token_id}""")
                    
        batch["labels"] = labels
        return batch

In [10]:
# "Instruct: <prompt>\nOutput:"
from datasets import load_dataset,DatasetDict
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import tqdm as notebook_tqdm

## Creating Dataset object and splitting them into train, test and valid

In [12]:
df=df[['text']]
# df = df.rename(columns={'generated_text':'text'})
dataset = ds.dataset(pa.Table.from_pandas(df))
# ### convert to Huggingface dataset
hg_dataset = Dataset(pa.Table.from_pandas(df))

In [13]:
train_testvalid = hg_dataset.train_test_split(test_size=0.2)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
train_test_valid_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

In [14]:
train_test_valid_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 40276
    })
    test: Dataset({
        features: ['text'],
        num_rows: 5035
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 5035
    })
})

## I am using the Unsloth Where I have mentioned the finetuning parameters

In [15]:
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state = 32,
    loftq_config = None,
)
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Unsloth 2025.3.19 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


Trainable: 5636096 | total: 1241450496 | Percentage: 0.4540%


## Defining the SFTTrainer

## Trained the model on two epochs which took 30+ hours to complete

In [16]:
import transformers
import torch
from transformers import TrainingArguments, TextStreamer
from trl import SFTTrainer,SFTConfig
max_seq_length = 8192
tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

training_arguments = SFTConfig(
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        dataset_num_proc = 2,
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        gradient_checkpointing=True,
        num_train_epochs=2,
        fp16=True,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        save_strategy="steps", 
        save_steps=50, 
        output_dir="/kaggle/working/Llama-3.2-1B-Instruct-SFT/output",
        seed=0,
        packing=False,
    )


trainer = SFTTrainer(
    model=model,
    train_dataset=train_test_valid_dataset["train"],
    eval_dataset=train_test_valid_dataset["valid"],
    # peft_config=lora_config,
    args=training_arguments,
    # Using custom data collator inside SFTTrainer
    data_collator=CustomDataCollatorWithPadding(
        tokenizer=tokenizer, 
        padding="longest", 
        max_length=max_seq_length, 
        return_tensors="pt"
    )
)





tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/40276 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/5035 [00:00<?, ? examples/s]

## Using wandb for getting the training progress

In [17]:
!wandb login --relogin 

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


## I have trained the model in multiple chunks

In [18]:
trainer.train(resume_from_checkpoint=True)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 40,276 | Num Epochs = 2 | Total steps = 628
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 4 x 1) = 128
 "-____-"     Trainable parameters = 5,636,096/1,000,000,000 (0.56% trained)
	save_steps: 50 (from args) != 500 (from trainer_state.json)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: iamarpanghosh (arpx) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
501,0.027600
502,0.027400
503,0.026900
504,0.027400
505,0.028000
506,0.029300
507,0.028100
508,0.029700
509,0.028100
510,0.028400


TrainOutput(global_step=628, training_loss=0.0055916392226602624, metrics={'train_runtime': 16344.6697, 'train_samples_per_second': 4.928, 'train_steps_per_second': 0.038, 'total_flos': 9.014416881376297e+17, 'train_loss': 0.0055916392226602624})

## Model is available here https://huggingface.co/Arpx22/llama-3.2-1B-Finetuned-ner-finance/tree/main/

In [23]:
trainer.push_to_hub()

adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

Upload 9 LFS files:   0%|          | 0/9 [00:00<?, ?it/s]

events.out.tfevents.1744348716.39b6fb311824.390.0:   0%|          | 0.00/72.2k [00:00<?, ?B/s]

events.out.tfevents.1744392197.e36a035d718e.119.0:   0%|          | 0.00/6.45k [00:00<?, ?B/s]

events.out.tfevents.1744348130.39b6fb311824.31.1:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

...

events.out.tfevents.1744392434.e36a035d718e.119.1:   0%|          | 0.00/65.2k [00:00<?, ?B/s]

events.out.tfevents.1744438599.c1d106e731ad.31.0:   0%|          | 0.00/33.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.69k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Arpx22/output/commit/f7e33713d8ee9354eb502743b1dbbcc1fbddb3b5', commit_message='End of training', commit_description='', oid='f7e33713d8ee9354eb502743b1dbbcc1fbddb3b5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Arpx22/output', endpoint='https://huggingface.co', repo_type='model', repo_id='Arpx22/output'), pr_revision=None, pr_num=None)

In [21]:
from huggingface_hub import notebook_login

In [22]:
notebook_login()

In [26]:
model.push_to_hub(repo_id = "Arpx22/llama-3.2-1B-Finetuned-ner-finance")
tokenizer.push_to_hub(repo_id = "Arpx22/llama-3.2-1B-Finetuned-ner-finance")

README.md:   0%|          | 0.00/577 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Arpx22/llama-3.2-1B-Finetuned-ner-finance


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

## Here is an example of testing

In [2]:
test_text = """Indian Overseas Bank reduces Repo Linked Lending Rate by 25 basis points
Indian Overseas Bank has lowered its Repo Linked Lending Rate by 25 basis points, effective immediately. This decision follows the Reserve B...

Delhi: Stampede-like situation reported at Indira Gandhi International Airport as 50 flights get delayed due to dust storm
A severe dust storm paralyzed Delhi and the NCR, causing widespread flight disruptions at Indira Gandhi International Airport. Over 50 domes...

India's auto component industry to hit USD 145 billion by 2030, exports to triple: NITI Aayog
NITI Aayog envisions India's automotive component production soaring to USD 145 billion by 2030, with exports tripling to USD 60 billion. A ...

Banks scouting buyers for Sahara Star Hotel loans
Lenders led by Union Bank of India are set to auction the Rs 700 crore debt owed by Sahara Star Hotel after the insolvency plea was rejected...

Global media titans to converge in Mumbai for first-ever WAVES summit in May
Mumbai is gearing up to host the inaugural World Audio Visual & Entertainment Summit (WAVES) from May 1-4, backed by the Indian government. ...

India is the place to be: Radisson Top Exec
Radisson Hotel Group is bullish on India's hotel development potential, aiming to reach 500 hotels by the end of the decade by signing 50 """

In [3]:
from  transformers import PreTrainedTokenizerBase
import json
def testing_prompt_xml(
    description_dict: dict, 
    input_str: str,
    tokenizer: PreTrainedTokenizerBase,
) :

    """
    Args:
    description (dict): dict of strings representing entity labels and its
                          corresponding description
    input_str (str): Actual input string on which detections need to be
                     performed
    label_str (str): Expected output string corresponding to input_str
    tokenizer (PreTrainedTokenizerBase): A tokenizer corresponding to the model
                                         being fine-tuned

    Returns: 
    torch.Tensor: Tensor of tokenized input ids
    """

    usr_msg1 = f"""You are given a user utterance that may contain Personal Identifiable Information (PII). 
        You are also given a list of entity types representing Personal Identifiable Information (PII). 
        Your task is to detect and identify all instances of the supplied PII entity types in the user utterance. 
        The output must have the same content as the input. Only the tokens that match the PII entities in the 
        list should be enclosed within XML tags. The XML tag comes from the PII entities described in the list below. 
        For example, a given name of a person should be enclosed within <FIRST_NAME></FIRST_NAME> tags.
        Ensure that all entities are identified. Do not perform false identifications.
        \n\nList Of Entities\n{json.dumps(description_dict,indent=2)}
        \n\n
        Are the instructions clear to you?"""
    
    asst_msg1 = """Yes, the instructions are clear. I will identify and enclose within the corresponding XML tags, 
        all instances of the specified PII entity types in the user utterance. For example, 
        <FIRST_NAME><Given name of a person></FIRST_NAME>, <PASSPORT_NUMBER><Number identifying a person’s passport></PASSPORT_NUMBER>, etc. 
        leaving the rest of the user utterance unchanged."""
    
    usr_msg2 = "My name is John Doe, and I can be contacted at meet_me_john@gmail.com"
    
    asst_msg2 = "My name is <FIRST_NAME>John</FIRST_NAME> <LAST_NAME>Doe</LAST_NAME>, and I can be contacted at <EMAIL>meet_me_john@gmail.com</EMAIL>"
    
    usr_msg3 = "Give a brief explanation of why your answer is correct."


    asst_msg3 = """My answer is correct because I identified the specified PII entity types in the user utterance and enclosed them within the corresponding XML tags.

                    - "John" is a given name, so it was enclosed within <FIRST_NAME> tags.
                    - "Doe" is a surname, so it was enclosed within <LAST_NAME> tags.
                    - "meet_me_john@gmail.com" is an email address, so it was enclosed within <EMAIL> tags."""
    
    usr_msg4 = """Great! I am now going to give you another user utterance. Please detect PII entities in it according to the previous instructions. Do not include an explanation in your answer."""
    
    asst_msg4 = "Sure! Please give me the user utterance."

    messages = [
        {"role": "user", "content": usr_msg1},
        {"role": "assistant", "content": asst_msg1},
        {"role": "user", "content": usr_msg2},
        {"role": "assistant", "content": asst_msg2},
        {"role": "user", "content": usr_msg3},
        {"role": "assistant", "content": asst_msg3},
        {"role": "user", "content": usr_msg4},
        {"role": "assistant", "content": asst_msg4},
        {"role": "user", "content": input_str},
    ]
    
    encoded_input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

    return encoded_input_ids

In [ ]:
model = FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    data_prompt.format(
        #instructions
        text,
        #answer
        "",
    )
], return_tensors = "pt").to("cuda")

In [1]:
# Model can be used from the cloud by this method
import unsloth
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Arpx22/llama-3.2-1B-Finetuned-ner-finance",
    max_seq_length=2048,
    load_in_4bit=True,  # Set to False if you prefer full precision
    dtype=None  ,        # Set to torch.float16 if needed
    gpu_memory_utilization=0.5
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.2.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 12.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

Unsloth 2025.3.19 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [4]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.2",  # or "chatml", "alpaca", etc. based on how you finetuned
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    map_eos_token=True
)

In [7]:
outputs = model.generate(testing_prompt_xml(entity_description_dict,test_text,tokenizer), max_new_tokens = 8192, use_cache = True)
answer=tokenizer.batch_decode(outputs)
print("Answer of the question is:", answer[0])

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Answer of the question is: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

You are given a user utterance that may contain Personal Identifiable Information (PII). 
        You are also given a list of entity types representing Personal Identifiable Information (PII). 
        Your task is to detect and identify all instances of the supplied PII entity types in the user utterance. 
        The output must have the same content as the input. Only the tokens that match the PII entities in the 
        list should be enclosed within XML tags. The XML tag comes from the PII entities described in the list below. 
        For example, a given name of a person should be enclosed within <FIRST_NAME></FIRST_NAME> tags.
        Ensure that all entities are identified. Do not perform false identifications.
        

List Of Entities
{
  "ACCOUNT_PIN": "Personal Identifi

In [8]:
import re
from typing import List, Tuple
# Helper Function for viewing the extracted entities
def extract_outermost_entities(text: str, valid_entity_types: List[str]) -> List[Tuple[str, str]]:
    entities = []
    
    for entity in valid_entity_types:
        # Match the outermost <ENTITY> ... </ENTITY> including malformed variants
        pattern = fr"""
            (<+{entity}>+)            # Opening tag (malformed tolerated)
            (.*?)                     # Non-greedy capture of content
            (</+{entity}>+)           # Closing tag (malformed tolerated)
        """
        # Use re.DOTALL to match across lines and VERBOSE for formatting
        matches = list(re.finditer(pattern, text, flags=re.DOTALL | re.VERBOSE))
        
        # To avoid nested matches, we skip overlaps
        used_spans = set()
        for match in matches:
            span = match.span()
            if any(start <= span[0] < end or start < span[1] <= end for start, end in used_spans):
                continue  # Skip if overlaps with already accepted match
            
            inner_text = match.group(2).strip()

            # Remove nested tags inside entity text
            inner_text = re.sub(r"<[^>]+>", "", inner_text).strip()
            cleaned_text = re.sub(r'{}'.format(entity), '', inner_text)
            cleaned_text = re.sub(r'[<>/\\\n\t]', '', cleaned_text)
            cleaned_text = cleaned_text.strip()
        
            # Deduplicate repeating sequences of words using regex
            pattern = r'\b(\w+(?:\s+\w+)*)\b(?:\s+\1\b)+'
            deduplicated_text = re.sub(pattern, r'\1', cleaned_text)
            
            entities.append((entity, deduplicated_text))
            used_spans.add(span)
    entities = list(set(entities))
    existing_ent = [('PASSPORT_NUMBER', ''),('LAST_NAME', 'Doe'),('FIRST_NAME', 'John'),('EMAIL', 'meet_me_john@gmail.com'),('FIRST_NAME', '')]
    entities = [ent for ent in entities if ent not in existing_ent]
    return list(set(entities))

In [9]:
entities_found = extract_outermost_entities(answer[0],list(entity_description_dict.keys()))

In [11]:
for ent in entities_found:
    print(ent)

('COMPANY', 'Radisson Hotel Group')
('COMPANY', 'Indian Over')
('DATE', 'May')
('DATE', 'May 1-4')
('DATE', 'the end of the decade')
('COMPANY', 'Union Bank of India')
('COMPANY', 'Indian Overseas Bank')
('DATE', '2030')
('COMPANY', 'NITI Aayog')


## The Model was trained only on two complete iteration and the generation take a lot of time (4 to 5 minutes for 1000 tokens) as my GPU is not that good 

In [12]:
import re
from typing import List, Tuple

def tag_entities_in_text(text: str, entities: List[Tuple[str, str]]) -> str:
    tagged_text = text
    used_spans = []

    # Sort entities to tag longer ones first to avoid substring overlaps
    for entity_type, entity_value in sorted(entities, key=lambda x: -len(x[1])):
        # Escape regex special characters in entity value
        escaped_value = re.escape(entity_value)

        # Build pattern to search the first unmatched occurrence
        pattern = re.compile(escaped_value)

        def replacement_fn(match):
            # Check if this span is already used
            span = match.span()
            if any(start < span[1] and span[0] < end for start, end in used_spans):
                return match.group(0)  # Don't tag again

            used_spans.append(span)
            return f"<{entity_type}>{match.group(0)}</{entity_type}>"

        # Only replace the first occurrence not already tagged
        tagged_text = pattern.sub(replacement_fn, tagged_text, count=1)

    return tagged_text


In [14]:
print(tag_entities_in_text(test_text,entities_found))

<COMPANY>Indian Overseas Bank</COMPANY> reduces Repo Linked Lending Rate by 25 basis points
Indian Overseas Bank has lowered its Repo Linked Lending Rate by 25 basis points, effective immediately. This decision follows the Reserve B...

Delhi: Stampede-like situation reported at Indira Gandhi International Airport as 50 flights get delayed due to dust storm
A severe dust storm paralyzed Delhi and the NCR, causing widespread flight disruptions at Indira Gandhi International Airport. Over 50 domes...

India's auto component industry to hit USD 145 billion by <DATE>2030</DATE>, exports to triple: <COMPANY>NITI Aayog</COMPANY>
NITI Aayog envisions India's automotive component production soaring to USD 145 billion by 2030, with exports tripling to USD 60 billion. A ...

Banks scouting buyers for Sahara Star Hotel loans
Lenders led by <COMPANY>Union Bank of India</COMPANY> are set to auction the Rs 700 crore debt owed by Sahara Star Hotel after the insolvency plea was rejected...

Global med

In [17]:
entities_found

[('COMPANY', 'Radisson Hotel Group'),
 ('COMPANY', 'Indian Over'),
 ('DATE', 'May'),
 ('DATE', 'May 1-4'),
 ('DATE', 'the end of the decade'),
 ('COMPANY', 'Union Bank of India'),
 ('COMPANY', 'Indian Overseas Bank'),
 ('DATE', '2030'),
 ('COMPANY', 'NITI Aayog')]